# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. 

В нашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Обучим на них разные модели для определения стоимости автомобиля и выберем одну с наилучшими значениями качества и скорости обучения, скорости предсказания.

**Цель исследования**: построить модель для определения стоимости автомобиля со значением RMSE-меры не менее 2500.  

**Ход исследования**

Данные для исследования получим из файла autos.csv. О качестве данных ничего не известно, поэтому перед обучением моделей понадобится их проверить. Найдем ошибки данных и оценим их влияние на исследование. На этапе подготовки попробуем исправить критичные из них.

Исследование пройдет в два этапа:
1. Изучение и подготовка данных.
2. Обучение и анализ моделей.

## Подготовка данных

Для начала импортируем необходимые в работе библиотеки.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OrdinalEncoder

Прочитаем файл `autos.csv` из каталога `datasets` и сохраним его в одноименной переменной.

In [2]:
autos = pd.read_csv('/datasets/autos.csv')

Составим первое впечатление о данных:
1. Получим общую информацию методом `info()`.
2. Выведем на экран первые пять строк таблицы методом `head()`. 
3. Оценим разброс значений методом `describe()`.

In [3]:
autos.info()
display(autos.head())
display(autos.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


Итак, в таблице 16 столбцов.
* DateCrawled — дата скачивания анкеты из базы
* Price — цена (евро)
* VehicleType — тип автомобильного кузова
* RegistrationYear — год регистрации автомобиля
* Gearbox — тип коробки передач
* Power — мощность (л. с.)
* Model — модель автомобиля
* Kilometer — пробег (км)
* RegistrationMonth — месяц регистрации автомобиля
* FuelType — тип топлива
* Brand — марка автомобиля
* NotRepaired — была машина в ремонте или нет
* DateCreated — дата создания анкеты
* NumberOfPictures — количество фотографий автомобиля
* PostalCode — почтовый индекс владельца анкеты (пользователя)
* LastSeen — дата последней активности пользователя


Качественные признаки `VehicleType`, `Gearbox`, `Model`, `FuelType`, `NotRepaired` содержат пропуски. Попробуем заполнить их на основе модальных значений признаков автомобилей со схожими параметрами (бренд и год регистрации транспортного средства). 

In [4]:
features_with_null = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired']

for col in features_with_null:
    autos.loc[autos[col].isna(), col] = autos.merge(autos.groupby(['Brand', 
              'RegistrationYear'])[col].apply(pd.Series.mode).reset_index(level=['Brand', 
              'RegistrationYear']), on=('Brand', 'RegistrationYear'), how='left')[col + '_y']

Проверим, остались ли пропуски в столбцах, функциями `isna()` и `sum()`.

In [5]:
autos[['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired']].isna().sum()

VehicleType    682
Gearbox         18
Model          186
FuelType        14
NotRepaired     67
dtype: int64

Оставшиеся пропуски в данных заполним модальным значением признака для соответствующего автомобильного бренда.

In [6]:
for col in features_with_null:
    autos.loc[autos[col].isna(), col] = autos.merge(autos.groupby('Brand')
          [col].apply(pd.Series.mode).reset_index(level='Brand'), 
          on='Brand', how='left')[col + '_y']

Избавимся от аномальных значений в столбцах `Price`, `RegistrationYear`, `Power` и `RegistrationMonth`. Найдем процентное соотношение аномалий в таблице. Так цена не может быть дешевле 100 евро, а мощность менее 45 л.с., месяц регистрации должен быть в диапазоне от 1 до 12, а год регистрации не меньше 1960 и не больше 2016, т.к. анкеты созданы не позднее этого года.

In [7]:
print(f'Процент аномальных значений признака Price: ' 
      f'{autos["Price"].value_counts(normalize=True)[lambda x: x.index < 100].sum():.1%}')

print(f'Процент аномальных значений признака RegistrationYear: ' 
      f'{autos["RegistrationYear"].value_counts(normalize=True)[lambda x: (x.index < 1960) | (x.index > 2016)].sum():.1%}')

print(f'Процент аномальных значений признака Power: ' 
      f'{autos["Power"].value_counts(normalize=True)[lambda x: x.index < 45].sum():.1%}')

print(f'Процент аномальных значений признака RegistrationMonth: ' 
      f'{autos["RegistrationMonth"].value_counts(normalize=True)[lambda x: (x.index < 1) | (x.index > 12)].sum():.1%}')

Процент аномальных значений признака Price: 3.8%
Процент аномальных значений признака RegistrationYear: 4.2%
Процент аномальных значений признака Power: 12.1%
Процент аномальных значений признака RegistrationMonth: 10.5%


Более 12% записей таблицы содержат аномалии в столбцах `Power` и `RegistrationMonth`. Слишком большой процент данных, чтобы избавляться от них. Мощность заменим медианными, а месяц регистрации модальными значениями признаков для автомобилей той же марки и года регистрации.

In [8]:
autos.loc[(autos['RegistrationMonth'] < 1) | (autos['RegistrationMonth'] > 12), 
          'RegistrationMonth'] = autos.merge(autos.groupby(['Brand', 
          'RegistrationYear'])['RegistrationMonth'].apply(pd.Series.mode).reset_index(level=['Brand', 
          'RegistrationYear']), on=('Brand', 'RegistrationYear'), how='left')['RegistrationMonth_y']

autos.loc[(autos['RegistrationMonth'] < 1) | (autos['RegistrationMonth'] > 12), 
          'RegistrationMonth'] = autos.merge(autos.groupby('Brand')
          ['RegistrationMonth'].apply(pd.Series.mode).reset_index(level='Brand'), 
          on=('Brand'), how='left')['RegistrationMonth_y']

autos.loc[autos['Power'] < 45, 'Power'] = autos.merge(autos.groupby(['Brand', 
          'RegistrationYear'])['Power'].median().reset_index(level=['Brand', 
          'RegistrationYear']), on=('Brand', 'RegistrationYear'), how='left')['Power_y']

autos.loc[autos['Power'] < 45, 'Power'] = autos.merge(autos.groupby('Brand')
          ['Power'].median().reset_index(level='Brand'), on=('Brand'), how='left')['Power_y']

Т.к. цена является целевой переменной, удалим записи с аномальным значением признака. Записи с некорректным годом регистрации также исключим из исследования. 

In [9]:
autos = autos.loc[(autos['Price'] >= 100) & (autos['RegistrationYear'] <= 2016)
                  & (autos['RegistrationYear'] >= 1960)]

Также избавимся от аномальных больших значений цены и мощности автомобилей, выходящие за значения трех стандартных отклонений.

In [10]:
autos = autos.loc[(autos['Power'] <= np.mean(autos['Power']) 
                  + 3 * np.std(autos['Power'])) & (autos['Price'] 
                  <= np.mean(autos['Price']) + 3 * np.std(autos['Price']))]

Даты скачивания и создания анкеты, дата последней активности и почтовый индекс пользователя являются уникальными для каждой записи таблицы. Исключим их из обучения моделей. Признак `NumberOfPictures` также удалим из таблицы, т.к. он является константным.

In [11]:
autos = autos.drop(['DateCrawled', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen'], axis=1)

### Выводы

В нашем распоряжении таблица autos, содержащая данные о технических характеристиках, комплектации и ценах автомобилей. На этапе предобработки были устранены пропуски в данных, скорректированы аномальные значения признаков.

После подготовки данных можно переходить к обучению моделей.

## Анализ моделей

Для решения задачи регрессии будем использовать три модели:
* линейная регрессия;
* случайный лес;
* градиентный бустинг.

Для обучения первых двух необходимо предварительно выполнить кодирование категориальных признаков. Сделаем копию таблицы `autos_ordinal` и с помощью метода порядкового кодирования `OrdinalEncoder` перекодируем признаки.

In [12]:
cat_features = ['VehicleType', 'Gearbox', 'Model', 'RegistrationMonth', 'FuelType', 'Brand', 'NotRepaired']

autos_ordinal = autos.copy()

encoder = OrdinalEncoder()
autos_ordinal[cat_features] = encoder.fit_transform(autos_ordinal[cat_features])

Целевым признаком нашей таблицы является столбец `Price`: запишем его в переменную `target`. Остальные признаки запишем в `features`.

In [13]:
features_ordinal = autos_ordinal.drop('Price', axis=1)
target_ordinal = autos_ordinal['Price']

Поделим исходный набор данных на обучающую и тестовую выборки в отношении 3:1 с помощью метода `train_test_split()`.

In [14]:
(features_ordinal_train, features_ordinal_test, 
 target_ordinal_train, target_ordinal_test) = train_test_split(features_ordinal, 
                                                               target_ordinal,
                                                               test_size=0.25,
                                                               random_state=777)

Проверим размеры полученных выборок функцией `shape`.

In [15]:
print(features_ordinal_train.shape, features_ordinal_test.shape)

(241634, 10) (80545, 10)


Определим лучшую модель случайного леса, изменяя значения гиперпараметров `n_estimators` и `max_depth`.

In [16]:
%%time
param_grid = {'n_estimators': range(10, 31, 10), 'max_depth': range(10, 31, 10)}
clf = (GridSearchCV(RandomForestRegressor(random_state=777),
                    param_grid, cv=5, scoring='neg_mean_squared_error'))
best_model = (clf.fit(features_ordinal_train, target_ordinal_train))
print(f'Лучшая модель на обучающей выборке: {best_model.best_estimator_}')

Лучшая модель на обучающей выборке: RandomForestRegressor(max_depth=20, n_estimators=30, random_state=777)
CPU times: user 6min 56s, sys: 5.07 s, total: 7min 1s
Wall time: 7min 1s


Обучим модель случайного леса с наилучшими гиперпараметрами, найденными на предыдущим этапе, и определим время обучения.

In [17]:
%%time
model = RandomForestRegressor(max_depth=20, n_estimators=30, random_state=777)
model.fit(features_ordinal_train, target_ordinal_train)

CPU times: user 17.7 s, sys: 140 ms, total: 17.8 s
Wall time: 17.8 s


RandomForestRegressor(max_depth=20, n_estimators=30, random_state=777)

Найдем значение среднеквадратичной ошибки RMSE на тестовой выборке.

In [18]:
%%time
probabilities_ordinal_test = model.predict(features_ordinal_test)
print(mean_squared_error(target_ordinal_test, probabilities_ordinal_test, squared=False))

1534.5693439711324
CPU times: user 739 ms, sys: 3.92 ms, total: 743 ms
Wall time: 742 ms


Значение RMSE-меры модели случайного леса на тестовой выборке равно 1534.6 при времени обучения около 18 секунд. 

Перейдем к обучению модели линейной регрессии. Для начала перекодируем качественные признаки методом прямого кодирования `OneHotEncoding`.

In [19]:
for col in cat_features:
    autos[col] = autos[col].astype('category')

autos_ohe = pd.get_dummies(autos, drop_first=True)

Выделим признаки и целевой признак таблицы, разделим данные на обучающую и тестовую выборки.

In [20]:
features_ohe = autos_ohe.drop('Price', axis=1)
target_ohe = autos_ohe['Price']

(features_ohe_train, features_ohe_test, 
 target_ohe_train, target_ohe_test) = train_test_split(features_ohe, 
                                                       target_ohe,
                                                       test_size=0.25,
                                                       random_state=777)

Обучим модель линейной регрессии на обучающей выборке и рассчитаем время обучения.

In [21]:
%%time
model = LinearRegression()
model.fit(features_ohe_train, target_ohe_train)

CPU times: user 19.5 s, sys: 10.6 s, total: 30.1 s
Wall time: 30.3 s


LinearRegression()

Модель линейной регрессии обучалась 30.3 секунд. Найдем RMSE-меру обученной модели на тестовой выборке.

In [22]:
%%time
probabilities_ohe_test = model.predict(features_ohe_test)
print(mean_squared_error(target_ohe_test, probabilities_ohe_test, squared=False))

2532.077856105489
CPU times: user 92.5 ms, sys: 87.6 ms, total: 180 ms
Wall time: 141 ms


Несмотря на более долгое время обучения, значение метрики RMSE модели линейной регрессии хуже, чем у модели случайного леса.

Перейдем к обучению модели градиентного бустинга библиотеки LightGBM. Для начала выделим из таблицы целевой признак `target` и признаки `features` и поделим данные на обучающую и тестовую выборки.

In [23]:
features = autos.drop('Price', axis=1)
target = autos['Price']

(features_train, features_test, 
 target_train, target_test) = train_test_split(features, 
                                               target,
                                               test_size=0.25,
                                               random_state=777)

Подберем наилучшие значения гиперпараметров `max_depth` и `num_leaves` модели методом `GridSearchCV()`.

In [24]:
%%time
param_grid = {'max_depth': range(10, 31, 10), 'num_leaves': range(3, 10, 3)}
clf = (GridSearchCV(LGBMRegressor(random_state=777),
                    param_grid, cv=5, scoring='neg_mean_squared_error'))
best_model = (clf.fit(features_train, target_train))
print(f'Лучшая модель на обучающей выборке: {best_model.best_estimator_}')

Лучшая модель на обучающей выборке: LGBMRegressor(max_depth=10, num_leaves=9, random_state=777)
CPU times: user 15min 37s, sys: 5.68 s, total: 15min 43s
Wall time: 15min 51s


Обучим модель градиентного бустинга с подобранными параметрами на обучающей выборке.

In [25]:
%%time
model = LGBMRegressor(max_depth=10, num_leaves=9, random_state=777)
model.fit(features_train, target_train)

CPU times: user 26.2 s, sys: 149 ms, total: 26.3 s
Wall time: 26.5 s


LGBMRegressor(max_depth=10, num_leaves=9, random_state=777)

Определим RMSE-меру модели обучения для тестовой выборки.

In [26]:
%%time
probabilities_test = model.predict(features_test)
print(mean_squared_error(target_test, probabilities_test, squared=False))

1676.7933623955807
CPU times: user 535 ms, sys: 2.74 ms, total: 538 ms
Wall time: 518 ms


Модель градиентного бустинга при скорости обучения 26.5 секунд смогла достичь на тестовой выборке значение RMSE = 1676.8.

### Выводы

В качестве моделей обучения были выбраны линейная регрессия, случайный лес и градиентный бустинг. Изменяя значения гиперпараметров мы смогли достичь на тестовой выборке лучшего значения RMSE-меры. 

Лучшее качество показала модель случайного леса, состоящая из 30 деревьев с глубиной дерева, равной 20 (RMSE = 1534.6). Модель линейной регресси и градиентного бустинга обучались дольше, но и значение RMSE-меры этих моделей оказалось выше.

## Общий вывод

На основе данных о технических характеристиках, комплектации и ценах автомобилей было проведено обучение трех моделей: линейная регрессия, случайный лес и градиентный бустинг для определения рыночной стоимости автомобиля. Для каждой модели были определены гиперпараметры, позволяющие добиться лучшего значения RMSE-метрики.

Наилучшее качество показала модель случайного леса, состоящая из 30 деревьев с глубиной дерева, равной 20. На тестовой выборке модель обучения достигла значения RMSE-меры = 1534.6, скорость обучения при этом 17.8 секунд. Модель градиентного бустинга с бОльшим значением квадратичной ошибки (RMSE = 1676.8) показала среднее время обучения 26.5 секунд. Модель линейной регрессии, обученная за полминуты, показала худшее качество RMSE = 2532.1. Время предсказания на тестовой выборке для модели случайного леса 742 милисекунды, для модели линейной регрессии: 141 милисекунд, для модели градиентного бустинга: 518 милисекунд.

Таким образом, выбирая модель машинного обучения для решения задачи исследования, особое внимание следует обратить на модель случайного леса.